
### 1. Import library

##### 1.1 - Import library

In [ ]:
import sqlite3
import pandas as pd

-------------------

### 2. Connect to Server and Create Database

##### 2.1 - Define Server Connection Function

**N.B. Update the variable 'pw' with the root password for your MySQL Server! Otherwise the connection cannot be made.**

In [ ]:
def create_db_connection(db_name):
    connection = None
    try:
        connection = sqlite3.connect(f"{db_name}.db")  # Connect or create the database file
        print(f"Connection to SQLite DB '{db_name}' successful.")
    except sqlite3.Error as e:
        print(f"The error '{e}' occurred.")
    return connection

##### 2.2 - Create a New Database


In [ ]:
db_name = "school"
connection = create_db_connection(db_name)

##### 2.3 - Define Query Execution Function

In [ ]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully.")
    except sqlite3.Error as e:
        print(f"The error '{e}' occurred.")

-------------------

### 3. Creating Tables

##### 3.1 - Create Teacher Table


In [ ]:
create_teacher_table = """
CREATE TABLE IF NOT EXISTS teacher (
  teacher_id INTEGER PRIMARY KEY,
  first_name TEXT NOT NULL,
  last_name TEXT NOT NULL,
  language_1 TEXT NOT NULL,
  language_2 TEXT,
  dob DATE,
  tax_id INTEGER UNIQUE,
  phone_no TEXT
);
"""

execute_query(connection, create_teacher_table)

##### 3.2 - Create Course Table

In [ ]:
create_course_table = """
CREATE TABLE course (
  course_id INT PRIMARY KEY,
  course_name TEXT NOT NULL,
  language TEXT NOT NULL,
  level TEXT,
  course_length_weeks INT,
  start_date DATE,
  in_school BOOLEAN,
  teacher INT,
  client INT
);
"""

execute_query(connection, create_course_table)

-----------------

### 4. Insert into Tables

##### 4.1 - Insert into Teacher Table


In [ ]:
teachers = """
INSERT INTO teacher VALUES
(1,  'James', 'Smith', 'ENG', NULL, '1985-04-20', 12345, '+491774553676'),
(2, 'Stefanie',  'Martin',  'FRA', NULL,  '1970-02-17', 23456, '+491234567890'),
(3, 'Steve', 'Wang',  'MAN', 'ENG', '1990-11-12', 34567, '+447840921333'),
(4, 'Friederike',  'Müller-Rossi', 'DEU', 'ITA', '1987-07-07',  45678, '+492345678901'),
(5, 'Isobel', 'Ivanova', 'RUS', 'ENG', '1963-05-30',  56789, '+491772635467'),
(6, 'Niamh', 'Murphy', 'ENG', 'IRI', '1995-09-08',  67890, '+491231231232');
"""

execute_query(connection, teachers)

##### 4.2 - Populate Course Table

In [ ]:
courses = """
INSERT INTO course VALUES
(12, 'English for Logistics', 'ENG', 'A1', 10, '2020-02-01', TRUE,  1, 105),
(13, 'Beginner English', 'ENG', 'A2', 40, '2019-11-12',  FALSE, 6, 101),
(14, 'Intermediate English', 'ENG', 'B2', 40, '2019-11-12', FALSE, 6, 101),
(15, 'Advanced English', 'ENG', 'C1', 40, '2019-11-12', FALSE, 6, 101),
(16, 'Mandarin für Autoindustrie', 'MAN', 'B1', 15, '2020-01-15', TRUE, 3, 103),
(17, 'Français intermédiaire', 'FRA', 'B1',  18, '2020-04-03', FALSE, 2, 101),
(18, 'Deutsch für Anfänger', 'DEU', 'A2', 8, '2020-02-14', TRUE, 4, 102),
(19, 'Intermediate English', 'ENG', 'B2', 10, '2020-03-29', FALSE, 1, 104),
(20, 'Fortgeschrittenes Russisch', 'RUS', 'C1',  4, '2020-04-08',  FALSE, 5, 103);
"""

execute_query(connection, courses)

--------------

### 5. Reading Data

##### 5.1 - Define Data Reading Function

In [ ]:
def read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as err:
        print(f"Error: '{err}'")

##### 5.2 - Read Data from Database

Let's try this with a simple query to begin with.

In [ ]:
q1 = """
SELECT *
FROM teacher;
"""
results = read_query(connection, q1)

for result in results:
  print(result)

And here are some more queries to try.

In [ ]:
q2 = """
SELECT last_name, dob
FROM teacher;
"""

q3 = """
SELECT *
FROM course
WHERE language = 'ENG'
ORDER BY start_date DESC;
"""

q4 = """
SELECT first_name, last_name, phone_no
FROM teacher
WHERE dob < '1990-01-01';
"""

q5 = """
SELECT course.course_id, course.course_name, course.language, teacher.first_name, teacher.last_name
FROM course
JOIN teacher
ON course.teacher = teacher.teacher_id
WHERE course.in_school = FALSE;
"""

queries = [q2, q3, q4, q5]

for q in queries:
  print(q)
  results = read_query(connection, q)
  for result in results:
    print(result)

##### 5.3 - Formatting Output into a pandas DataFrame


In [ ]:
from_db = []

results = read_query(connection, q5)
for result in results:
  result = list(result)
  from_db.append(result)


columns = ["course_id", "course_name", "language",
           "teacher_first_name", "teacher_last_name"]
df = pd.DataFrame(from_db, columns=columns)

display(df)

### 6. Updating Records

##### 6.1 - Updating Teacher name

In [ ]:
update = """
UPDATE teacher
SET first_name = 'Jim', last_name = 'Halpert'
WHERE teacher_id = 1;
"""

execute_query(connection, update)

Let's see if that worked.

In [ ]:
q1 = """
SELECT *
FROM teacher
WHERE teacher_id = 1;
"""

results = read_query(connection, q1)

for result in results:
  print(result)

### 7. Deleting Records

##### 7.1 - Deleting a Course

In [ ]:
q1 = """
SELECT *
FROM course;
"""

results = read_query(connection, q1)

from_db = []

for result in results:
  print(result)

Let's delete the course with course_id 20 - 'Fortgeschrittenes Russisch'. For this we will use the [DELETE](https://dev.mysql.com/doc/refman/8.0/en/delete.html) SQL command.

In [ ]:
delete_course = """
DELETE FROM course WHERE course_id = 20;
"""

execute_query(connection, delete_course)

Let's confirm that the course is gone.

In [ ]:
q1 = """
SELECT *
FROM course;
"""

results = read_query(connection, q1)

from_db = []

for result in results:
  print(result)

### 8. CSV to SQL

In [ ]:
!wget -O teacher_data.csv 'https://www.dropbox.com/scl/fi/kop7h5r7ilell91j3n6n2/Updated_Teacher_Data_CSV.csv?rlkey=d5xnwmbacxrlrt1bppjds2842&st=axph0s1v&dl=0'

In [ ]:
teacher_df = pd.read_csv("teacher_data.csv")

cursor = connection.cursor()

for index, row in teacher_df.iterrows():
  sql = """INSERT INTO teacher (teacher_id, first_name, last_name, language_1, language_2, dob, tax_id, phone_no)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?)"""
  print(index, tuple(row))
  print(sql)
  print()
  cursor.execute(sql, tuple(row))


In [ ]:
query = "SELECT * FROM teacher;"
df_teacher_from_db = pd.read_sql_query(query, connection)
df_teacher_from_db